In [1]:
# 这个文件，用于生成以地址为粒度的数据

In [2]:
from sklearn.feature_extraction import DictVectorizer
from tqdm import tqdm
import csv
import time
from datetime import datetime
from sklearn import tree
from sklearn import preprocessing
from six import StringIO
import json
import pandas as pd
import os
from prettytable import PrettyTable
import redis
import json

In [3]:
db = redis.Redis(host= "localhost",port= 6379,decode_responses= True)

In [4]:
# path = r"/Users/linke/Desktop/myspace/git/2016/"
path = r"R:/paper/trans/2016/"

# path = r"/Users/linke/Desktop/myspace/git/ransomware/ransomware/ransomware/ransomwareTxs"

# production_path = r"/Users/linke/Desktop/myspace/git/ransomware/ransomware/production/noraml_data.csv"
# production_path = r"/Users/linke/Desktop/myspace/git/ransomware/ransomware/production/noraml_data.csv"
production_path = r"R:/paper/git/ransomware/production/noraml_data.csv"

ransom_addrs = []

In [5]:
def diff_day(t1, t2):
    dt1 = datetime.utcfromtimestamp(t1)
    dt2 = datetime.utcfromtimestamp(t2)
    return (dt2 - dt1).days

In [6]:
def analysize_timestamps(ts0, ts1, ts2):
    result = []
#     result.append(diff_day(max(ts1), min(ts2)))

    _all = ts1 + ts2
    _all.sort()
    result.append(diff_day(_all[0], _all[-1]))

    day = lambda ts : datetime.utcfromtimestamp(ts).date()
    days = [day(ts) for ts in _all]
    day_num = pd.value_counts(days)
    for ts in ts0:
        day_num[day(ts)] -= 1
    result.append(len(day_num))
    result.append(day_num.max())

    if len(ts1) == 0:
        result.append(0)
    else:
        result.append(1 + diff_day(min(ts1), max(ts1)))
    if len(ts2) == 0:
        result.append(0)
    else:   
        result.append(1 + diff_day(min(ts2), max(ts2)))

    return result

In [7]:
def read_csv_to_list(path):
    messages = []
    
    for root,dirs,files in os.walk(path):
        for dir in dirs:
            for r,ds,fs in os.walk(dir):
                for f in fs:
                    readFile(messages, r, f)
        for file in files:
            readFile(messages, root, file)

    return messages

In [8]:
def getAddrName(str):
    return str.split('_txs.json')[0]

In [9]:
def readFile(messages, rootName, fileName):
    try:
        addrName = getAddrName(fileName)
#         ransom_addrs.append(addrName)
        
        f = open(os.path.join(rootName, fileName))
        line = f.readline()
        while line:
            transaction_json = json.loads(line)
            messages.append(transaction_json)
            line = f.readline()
    except Exception as ex:
        print(fileName)

    f.close()

In [10]:
# trans = {一系列交易}
# addr_trans = {某一个地址: [ [相关事务id], [传入交易时间], [传出交易时间], 传入数量, 传入金额, 传出数量, 传出金额, 
#                          存在自己传给自己的现象的事务数量, 自己传给自己的现象的事务的金额总数, [自传自事务时间]]
#               第二个地址: [...]}

#[[事务id], [传入数量， 传入金额, 传入交易时间], [传出数量, 传出金额, 传出交易时间], [自传自数量, 金额, 时间]]

In [11]:
def rw_db(addr, index, value):
    msg = eval(db.get(addr))
    msg[index].append(value)
    db.set(addr, json.dumps(msg))

In [ ]:
trans = read_csv_to_list(path)

trash_pool = []

for tran in tqdm(trans):
    if tran in trash_pool:
        continue
    trash_pool.append(tran)
    
    in_addr = []
    out_addr = []
    
    for vin in tran['vin']:
        if 'address' not in vin:
            continue
            
        addr = vin['address']
        msg = db.get(addr)
        if msg == None:
            msg = [[],[],[],0,0, 0,0, 0,0, []]            
        else:
            msg = eval(msg)
        
        msg[0].append(tran['txhash'])  
        msg[3] += 1
        msg[4] += vin['value']
        if addr not in in_addr:
            in_addr.append(addr)
        db.set(addr, json.dumps(msg))
        
    for vout in tran['vout']:
        addr = vout['address']
        msg = db.get(addr)
        if msg == None:
            msg = [[],[],[],0,0, 0,0, 0,0, []]
        else:
            msg = eval(msg)

        msg[0].append(tran['txhash'])
        msg[5] += 1
        msg[6] += vout['value']
        
        if addr not in out_addr:
            out_addr.append(addr)
            
        if addr in in_addr:
            msg[7] += 1
            msg[8] += vout['value']
        
        db.set(addr, json.dumps(msg))
            
     
    for addr in in_addr:
        rw_db(addr, 1, tran['timestamp'])
    for addr in out_addr:
        rw_db(addr, 2, tran['timestamp'])
    for addr in in_addr:
      if addr in out_addr:
        rw_db(addr, 9, tran['timestamp'])

 25%|█████████████████▉                                                       | 46386/188329 [15:09<1:27:59, 26.89it/s]

In [ ]:
# 地址： 涉及交易数量, 传入数量, 传入金额, 传出数量, 传出金额, 存在自己传给自己的现象的事务数量, 自己传给自己的现象的事务的金额总数,
# (最晚输入与最早输出天数之差), 生命周期, 活动天数, 每日最大交易次数, 传入时间的区间长度, 传出时间的区间长度

In [ ]:
addr_trans = {}
for addr in trash_pool:
    addr_trans[addr] = eval(db.get(addr))

In [ ]:
table = []
for key in tqdm(addr_trans):
    temp = []
    temp.append(key)
    temp.append(1 if key in ransom_addrs else 0)
    temp.append(len(addr_trans[key][0]))
    temp.append(addr_trans[key][3])
    temp.append(addr_trans[key][4])
    temp.append(addr_trans[key][5])
    temp.append(addr_trans[key][6])
    temp.append(addr_trans[key][7])
    temp.append(addr_trans[key][8])
    
    time_data = analysize_timestamps(addr_trans[key][9], addr_trans[key][1], addr_trans[key][2])
    temp.extend(time_data)
    
    table.append(temp)

In [ ]:
data = pd.DataFrame(table)
data.columns = ['address','is_ransom','trans_num', 'in_num', 'in_coins', 'out_num', 'out_coins', 'exchanges', 'ex_coins', 'lifetime', 'active_days', 'most_perday', 'in_time_len', 'out_time_len']
data.to_csv(production_path, index = False)